In [1]:
%matplotlib inline
import scipy
import gempy as gp
import geopandas as gpd
import os
import pyvista 
import rasterio
import gemgis as gg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyvista import set_plot_theme
import matplotlib.image as mpimg

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\morit\anaconda3\envs\gemgis\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data_path= 'https://raw.githubusercontent.com/Mo34843279347/Sandstone_Mountains/main/'
geo_model = gp.create_model('ES_model')
geo_model = gp.init_data(geo_model, extent=[0, 1000, 0, 490, -400, 500], resolution=[100, 10, 100],
                        path_o= data_path+ 'orientations.csv',
                        path_i= data_path+ 'surface_points.csv')
gp.set_interpolator(geo_model, theano_optimizer='fast_compile', verbose=[])
gp.map_stack_to_surfaces(geo_model, {'Fault':'Fault'})
geo_model.reorder_features(['Fault','Default series', 'Basement'])

Active grids: ['regular']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                   values
range            1431.82
$C_o$           48811.90
drift equations   [3, 3]


,order_series,BottomRelation,isActive,isFault,isFinite
Fault,1,Erosion,True,False,False
Default series,2,Erosion,True,False,False
Basement,3,Erosion,False,False,False


In [3]:
geo_model.set_topography(source='random', fd=1.9, d_z=np.array([0, 250]),
                         resolution=np.array([100, 49]))
gp.set_interpolator(geo_model)
gp.compute_model(geo_model, compute_mesh=False, set_solutions=True)
geo_model._grid.topography.save('Random_topo')
geo_model.surfaces

Active grids: ['regular' 'topography']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                     values
range              1431.82
$C_o$             48811.90
drift equations  [3, 3, 3]


C:\Users\morit\anaconda3\envs\gemgis\lib\site-packages\gempy\core\solution.py:174: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.geological_map = np.array(


,surface,series,order_surfaces,color,id
5,Fault,Fault,1,#ff3f20,1
4,surface5,Default series,1,#443988,2
3,surface4,Default series,2,#728f02,3
0,surface1,Default series,3,#015482,4
2,surface3,Default series,4,#ffbe00,5
1,surface2,Default series,5,#9f0052,6
6,basement,Basement,1,#5DA629,7


# Topography

In [4]:
raster_original = np.genfromtxt('Raster.csv', delimiter=',')
new_topo= np.load('Random_topo.npy')
np.shape(raster_original)

(490, 1000)

In [6]:
#set height values from orignal raster to random topo
raster_original_cut= raster_original[::10,::10]
height= np.reshape(raster_original_cut, (100,49,1))
new_topo[:,:,2:3]= height
np.save('raster_original_cut',new_topo)


In [7]:
#set new topography
geo_model.set_topography(source='saved', filepath='raster_original_cut.npy')
gp.set_interpolator(geo_model)
gp.compute_model(geo_model, compute_mesh=False, set_solutions=True)

gp.compute_model(geo_model)
geo_model.additional_data.kriging_data
geo_model.additional_data.rescaling_data

#show model
g3d = gp.plot_3d(geo_model,
                 plotter_type='background',
                 show_topography=True,
                 show_lith=True,
                 show_surfaces=True,
                 kwargs_plot_structured_grid={'opacity': .15},
                 ve=1, 
                 off_screen=False)

Active grids: ['regular' 'topography']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                     values
range              1431.82
$C_o$             48811.90
drift equations  [3, 3, 3]


C:\Users\morit\anaconda3\envs\gemgis\lib\site-packages\gempy\core\solution.py:174: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.geological_map = np.array(
C:\Users\morit\anaconda3\envs\gemgis\lib\site-packages\pyvista\plotting\plotting.py:1354: PyvistaDeprecationWarning: 
"stitle" is a depreciated keyword and will be removed in a future
release.

Use ``scalar_bar_args`` instead.  For example:

scalar_bar_args={'title': 'Scalar Bar Title'}

  warnings.warn(USE_SCALAR_BAR_ARGS, PyvistaDeprecationWarning)
